In [1]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm


def extract_date(text):
    # Helper function to extract date in the format dd/mm/yyyy
    match = re.search(r'(\d{2}/\d{2}/\d{4})', text)
    if match:
        return pd.to_datetime(match.group(1), dayfirst=True)
    return None

def extract_disembark_loc(text):
    # Helper function to extract the disembark location from text and clean it up
    match = re.search(r'débarqué[es]? à ([A-Za-zÀ-ÿ\'-]+(?: [A-Za-zÀ-ÿ\'-]+)*)', text, flags=re.IGNORECASE)
    if match:
        place_name = match.group(1)
        # Remove trailing non-place words like "le", "la", "du", etc.
        place_name = re.sub(r'\b(?:le|la|du|en|aux)\b.*$', '', place_name, flags=re.IGNORECASE).strip()
        return place_name
    return None

def split_remarks(remarks):
    # Split remarks based on 'rembarqué' keyword to handle each segment separately
    parts = re.split(r'(rembarqué[es]?)', remarks, flags=re.IGNORECASE)
    segments = [parts[0]]  # First part before 'rembarqué'
    
    for i in range(1, len(parts) - 1, 2):
        segments.append(parts[i] + parts[i + 1])  # Combine 'rembarqué' with the following text
    
    return segments

def process_rembarque(df):
    new_rows = []

    # Filter rows that contain the "rembarqué" keyword in the Remarks column
    rembarque_rows = df[df['Remarks'].str.contains('rembarqué', na=False, case=False)]

    for index, row in rembarque_rows.iterrows():
        segments = split_remarks(row['Remarks'])

        # Process the first segment (before the first rembarqué)
        first_segment = segments[0]
        first_embark_date = row['Emb_date']  # Use original embark date
        first_disembark_date = extract_date(re.search(r'débarqué[es]?.*', first_segment, flags=re.IGNORECASE).group()) if re.search(r'débarqué[es]?.*', first_segment, flags=re.IGNORECASE) else None
        first_disembark_loc = extract_disembark_loc(first_segment)  # Extract disembark location

        original_row = row.copy()
        original_row['Emb_date'] = first_embark_date  # Keep original embark date
        original_row['Disemb_date'] = first_disembark_date if first_disembark_date else row['Disemb_date']
        original_row['Disemb_loc'] = first_disembark_loc if first_disembark_loc else row['Disemb_loc']
        original_row['Remarks'] = first_segment.strip()
        new_rows.append(original_row)

        previous_disembark_date = first_disembark_date

        # Process subsequent segments starting with "rembarqué"
        for i, segment in enumerate(segments[1:], start=1):
            # Handle "rembarqué le dit jour et an"
            if 'rembarqué le dit jour et an' in segment.lower():
                embark_date = previous_disembark_date
            else:
                embark_date = extract_date(re.search(r'rembarqué[es]?.*', segment, flags=re.IGNORECASE).group()) if re.search(r'rembarqué[es]?.*', segment, flags=re.IGNORECASE) else None

            disembark_date = extract_date(re.search(r'débarqué[es]?.*', segment, flags=re.IGNORECASE).group()) if re.search(r'débarqué[es]?.*', segment, flags=re.IGNORECASE) else None
            disembark_loc = extract_disembark_loc(segment)  # Extract disembark location

            new_row = row.copy()
            new_row['Emb_date'] = embark_date
            new_row['Disemb_date'] = disembark_date if disembark_date else row['Disemb_date']
            
            # Only the last segment should keep the original Disemb_loc; others should extract it
            if i == len(segments) - 1:
                new_row['Disemb_loc'] = row['Disemb_loc']
            else:
                new_row['Disemb_loc'] = disembark_loc if disembark_loc else row['Disemb_loc']

            new_row['Remarks'] = segment.strip()
            new_rows.append(new_row)

            # Update the previous disembark date
            previous_disembark_date = disembark_date if disembark_date else previous_disembark_date

    # Append the processed rows back to the original dataframe
    non_rembarque_rows = df[~df.index.isin(rembarque_rows.index)]
    new_df = pd.concat([non_rembarque_rows, pd.DataFrame(new_rows)], ignore_index=True)

    # Sort the dataframe by Last Name and Embark Date
    new_df = new_df.sort_values(by=['Last Name', 'Emb_date'])

    return new_df

def replace_nan_array(val):
    if isinstance(val, np.ndarray):
        if val.size == 1 and np.isnan(val[0]):
            return np.nan
    return val

def match_or_nan(a, b):
    # Function to check if two values are equal or both are NaN
    return (a == b) | (pd.isna(a) & pd.isna(b))

def check_issues(row):
    issues = []
    if not pd.isna(row['Emb_date']) and not pd.Series(row['Emb_date']).str.match(r'^\d{2}/\d{2}/\d{4}$', na=False).bool():
        issues.append('emb_date')
    if pd.isna(row['Emb_date']):
        issues.append('emb_date')
    if not pd.isna(row['Disemb_date']) and not pd.Series(row['Disemb_date']).str.match(r'^\d{2}/\d{2}/\d{4}$', na=False).bool():
        issues.append('disemb_date')
    if pd.isna(row['Disemb_date']):
        issues.append('Disemb_date')
    if pd.isna(row['Emb_loc']):
        issues.append('emb_loc')
    if pd.isna(row['Disemb_loc']):
        issues.append('disemb_loc')
    return issues


def convert_to_ddmmyyyy(date_value):
    if pd.isna(date_value) or str(date_value).lower() in ['nan', '[nan]']:
        return np.nan
    try:
        # Parse the date and reformat to dd/mm/yyyy
        return pd.to_datetime(date_value).strftime('%d/%m/%Y')
    except (ValueError, TypeError):
        return date_value


def extract_details(text):
    # Patterns for embarkation
    embark_patterns = {
        'embarqué': {
            'date': r'\b(?:embarqué|rembarqué|embarquée|rembarquée|embarqués|rembarqués|embarquées|rembarquées) (?:(?!\b(?:débarqué|noyé(?:e|es|s)?|débarquée|débarqués|débarquées|déserté|désertée|désertés|désertées|mort|morte|morts|mortes|passé|passée|passés|passées|resté)\b).)*(?:le|du) (\d{2}/\d{2}/\d{4})(?=\s|\n|$| ---|,)',
            'location': r'\b(?:embarqué|rembarqué|embarquée|rembarquée|embarqués|rembarqués|embarquées|rembarquées) (?:comme mousse à|en remplacement à|clandestinement au départ de|clandestinement à|furtivement à|furtivement au|au|à|en supplément à|en) ([\w\s\'-îÎ]+?)(?=\s+\b(?:pour la|départ|fin|mort(?:e|es|s)?|débarqué(?:e|es|s)?|déserté(?:e|es|s)?|passé(?:e|es|s)?|resté(?:e|es|s)?|tombé(?:e|es|s)?|repassé(?:e|es|s)?)\b|en remplacement|en supplément| (?:le|du) \d{2}/\d{2}/\d{4}|,| ---|\n|$| a servi à bord)'
        },
#         'a fait la': {
#             'date': r'remplacement (?:le|du) (\d{2}/\d{2}/\d{4})',
#             'location': r'\b(?:a|à) fait la campagne de ([\w\s\'-îÎ]+?) à ([\w\s\'-îÎ]+)'
#         },
        'supplément': {
            'date': r'\bsupplément (?:à|au) (?!\b(?:débarqué|débarquée|débarqués|débarquées|déserté|désertée|désertés|désertées|mort|morte|morts|mortes|passé|passée|passés|passées)\b)[\w\s\'-îÎ]+? (?:du|le) (\d{2}/\d{2}/\d{4})(?=\s|\n|$| ---|,)',
            'location': r'\bsupplément (?:à|au) ([\w\s\'-îÎ]+?)(?=\s|\n|$|(?:du|le) \d{2}/\d{2}/\d{4}| ---|,)'
        },
        'complément': {
            'date': r'\bcomplément (?:à|au) (?!\b(?:débarqué|débarquée|débarqués|débarquées|déserté|désertée|désertés|désertées|mort|morte|morts|mortes|passé|passée|passés|passées)\b)[\w\s\'-îÎ]+? (?:du|le) (\d{2}/\d{2}/\d{4})(?=\s|\n|$| ---|,)',
            'location': r'\bcomplément (?:à|au) ([\w\s\'-îÎ]+?)(?=\s|\n|$|(?:du|le) \d{2}/\d{2}/\d{4}| ---|,)'
        },
        'remplacement': {
            'date': r'\b(?:remplacement|remplaçement) (?:au|à|en)?(?!\b(?:débarqué|débarquée|débarqués|débarquées|déserté|désertée|désertés|désertées|mort|morte|morts|mortes|passé|passée|passés|passées)\b)[\w\s\'-îÎ]*?(?:le|du) (\d{2}/\d{2}/\d{4})',
            'location': r'\b(?:remplacement|remplaçement) (?:en rade de|au|à|en)? ([\w\s\'-îÎ]+?)(?= débarqué(?:s|es|s)?| déserté(?:s|es|s)?| mort(?:s|es|s)?| passé(?:s|es|s)?| en| le| du|\n|,| ---|$)'
        },
        'embanremp': {
            'date': r'\bembarqué en remplacement (?:au|à)?(?!\b(?:débarqué|débarquée|débarqués|débarquées|déserté|désertée|désertés|désertées|mort|morte|morts|mortes|passé|passée|passés|passées)\b)[\w\s\'-îÎ]*?(?:le|du) (\d{2}/\d{2}/\d{4})',
            'location': r'\bembarqué en remplacement (?:au|à)? ([\w\s\'-îÎ]+?)(?= débarqué(?:s|es|s)?| déserté(?:s|es|s)?| mort(?:s|es|s)?| passé(?:s|es|s)?| en| le| du|\n|,| ---|$)'
        },
        'trouvé': {
            'date': r'\btrouvé (?:(?!\b(?:quitté(?:e|es|s)?|matelot|débarqué|noyé(?:e|es|s)?|débarquée|débarqués|débarquées|déserté|désertée|désertés|désertées|mort|morte|morts|mortes|passé|passée|passés|passées|resté)\b).)*\s*(?:le|du) (\d{2}/\d{2}/\d{4})(?=\s|\n|$| ---|,)',
            'location': r'\btrouvé(?:e|es|s)? (?:à bord au départ du cap de|à bord au départ du|à bord en mer|à bord en partant de|à bord en partant du|à bord en quittant le|à bord au moment du départ de|à bord au moment du départ|à bord du vaisseau après le départ de|à bord après le départ de|à bord après le départ|dans le vaisseau au départ de|dans le vaisseau|à bord du vaisseau|caché(?:e|es|s)? au départ de|caché(?:e|es|s)? à bord au départ de|caché(?:e|es|s)? à bord au départ|caché(?:e|es|s)? à bord|caché(?:e|es|s)? à|à bord au départ de|à bord à|à bord|au départ de|à|au) (?!\s*(?:le|du)\b)([\w\s\'-îÎ]+?)?(?=\s*(?:le|du) \d{2}/\d{2}/\d{4}|a servi|---|,|\n|$|\b(?:débarqué(?:e|es|s)?|déserté(?:e|es|s)?|mort(?:e|es|s)?|passé(?:e|es|s)?))'
        }, 
        'soldat': {
            'date': r'\bsoldat(?:e|es|s)? à bord le\s*(\d{2}/\d{2}/\d{4})(?:\s|,| ---|$)',
            'location': r''
        },
        'né': {
            'date': r'\bné(?:e|es|s)? (?:à bord|en mer) (?:le|du) (\d{2}/\d{2}/\d{4})',
            'location': r''
        }
    }

    # Patterns for disembarkation
    disembark_patterns = {
        'débarqué': {
            'date': r'\bdébarqué(?:e|es|s)? (?:et passé(?:es|s|e)? aux travaux (?:à|de|du|au|en)|et resté(?:es|s|e)? contre son gré (?:à|de|du|au|en)|et déserté (?:à|de|du|au|en)|scorbutique (?:à|de|du|au|en)|malade et resté(?:e|es|s)? estropié(?:e|es|s)? au combat (?:à|de|du|au|en)|par échange(?:e|es|s)? (?:à|de|du|au|en)|grièvement blessé(?:e|es|s)? (?:à|de|du|au|en)|attaqué(?:e|es|s)? de folie (?:à|de|du|au|en)|et passé(?:e|es|s)? aux forges (?:à|de|du|au|en)|et vendu (?:à|de|du|au|en)|à l\'hôpital à la relâche (?:à|de|du|au|en)|et resté(?:e|s|es)? employé(?:e|s|es)? (?:à|de|du|au|en)|et parti pour aller (?:à|de|du|au|en)|pour aller (?:à|de|du|au|en)|et resté(?:e|s|es)? officier des troupes (?:à|de|du|au|en)|malade à la relache (?:à|de|du|au|en)|au comptoir (?:à|de|du|au|en)|et resté(?:e|s|es)? chez son oncle (?:à|de|du|au|en)|pour l\'hôpital (?:à|de|du|au|en)|par ordre du Commandant (?:à|de|du|au|en)|par ordre (?:à|de|du|au|en)|de gré à l\'hôpital (?:à|de|du|au|en)|et resté(?:e|es|s)? à l\'hôpital (?:à|de|du|au|en)|avec ses effets (?:à|de|du|au|en)|et resté(?:e|es|s)? à ses frais (?:à|de|du|au|en)|et resté(?:e|es|s)? malade à l\'hôpital (?:à|de|du|au|en)|et resté(?:e|es|s)? aumônier à l\'hôpital (?:à|de|du|au|en)|avec ses hardes, de gré à gré (?:à|de|du|au|en)|avec ses hardes (?:à|de|du|au|en)|ou resté(?:e|es|s)? (?:à|de|du|au|en)|et engagé(?:e|es|s)? dans la troupe (?:à|de|du|au|en)|dans la|et engagé(?:e|es|s)? dans le régiment (?:à|de|du|au|en)|et engagé(?:e|es|s)? dans|et engagé(?:e|es|s)? (?:à|de|du|au|en)|pour la traite (?:à|de|du|au|en)|et retenu (?:à|de|du|au|en)|pour maladie (?:à|de|du|au|en)|pour affaires de famille (?:à|de|du|au|en)|à malade à l\'hôpital (?:à|de|du|au|en)|et revenu sur le|malade dans le bateau du pilote (?:à|de|du|au|en)|malade dans le|et incorporé(?:e|es|s)? à la garnison (?:à|de|du|au|en)|et incorporé(?:e|es|s)? dans la garnison (?:à|de|du|au|en)|pour le service (?:à|de|du|au|en)|malade et resté(?:e|es|s)? (?:à|de|du|au|en)|de bonne volonté (?:à|de|du|au|en)|et mis à bord (?:à|de|du|au|en)|malade à la relâche (?:à|de|du|au|en)|à la relâche (?:à|de|du|au|en)|sans ordre (?:à|de|du|au|en)|et resté(?:e|es|s)? comme matelot (?:à|de|du|au|en)|malade à l\'hôpital du roi (?:à|de|du|au|en)|à l\'hôpital du roi (?:à|de|du|au|en)|au bas (?:à|de|du|au|en)|et (?:congédié(?:e|es|s)?|soldé(?:e|es|s)?) (?:à|de|du|au|en)|et remis sur le fort (?:à|de|du|au|en)|et remis (?:à|de|du|au|en)|pour se rendre (?:à|de|du|au|en)|blessé(?:e|s|es)? malade (?:à|de|du|au|en)|blessé(?:e|s|es)? (?:à|de|du|au|en)|de gré pour le service (?:à|de|du|au|en)|de gré à gré (?:à|de|du|au|en)|puis passé(?:e|es|s)? malade (?:à|de|du|au|en)|et resté(?:e|es|s)? (?:à|de|du|au|en)|et passé(?:e|es|s)? sur (?:le|la)|et passé(?:e|es|s)? (?:à|de|du|au|en)|et entré(?:e|es|s)? aux forges (?:à|de|du|au|en)|et mis au (?:prison|cachot) (?:à|de|du|au|en)|de son gré (?:à|de|du|au|en)|malade d\'une descente à la relâche (?:à|de|du|au|en)|malade du scorbut (?:à|de|du|au|en)|et resté(?:e|es|s)? (?:à|de|du|au|en)|dans le|et renvoyé(?:e|es|s)? (?:à|de|du|au|en)|et entré(?:e|es|s)? à l\'hôpital (?:à|de|du|au|en)|et payé(?:e|es|s)? (?:à|de|du|au|en)|de gré (?:à|de|du|au|en)|et resté(?:e|es|s)? à terre|au service du roi (?:à|de|du|au|en) |à l\'hôpital (?:à|de|du|au|en) |à |au |en échange (?:à|de|du|au|en) |furtivement (?:à|de|du|au|en) |malade après être tombé du haut mal (?:à|de|du|au|en) |malade à l\'hôpital (?:à|de|du|au|en) |malade (?:à|en|au|du|de) |malade et mort(?:e|es|s)? à l\'hôpital (?:à|de|du|au|en) |malade (?:à|de|du|au|en) |à malade (?:à|de|du|au|en) |à |au |en |de |aux |malade |pour )?[^\d\n,]*?(?:le|du) (\d{2}(?:-|/)\d{2}/\d{4})(?=(\s*---|\n|$|\s*passé(?:e|es|s)?(?:s|e|es)?|\s*échappé(?:e|s|es)?|\s*mort(?:e|es|s)?(?:e|s|es)?|\s*noyé(?:e|s|es)?))',
            'location': r'\bdébarqué(?:e|es|s)? (?:et passé(?:es|s|e)? aux travaux (?:à|de|du|au|en)|et resté(?:es|s|e)? contre son gré (?:à|de|du|au|en)|et déserté (?:à|de|du|au|en)|scorbutique (?:à|de|du|au|en)|malade et resté(?:e|es|s)? estropié(?:e|es|s)? au combat (?:à|de|du|au|en)|par échange(?:e|es|s)? (?:à|de|du|au|en)|grièvement blessé(?:e|es|s)? (?:à|de|du|au|en)|attaqué(?:e|es|s)? de folie (?:à|de|du|au|en)|et passé(?:e|es|s)? aux forges (?:à|de|du|au|en)|et vendu (?:à|de|du|au|en)|à l\'hôpital à la relâche (?:à|de|du|au|en)|et resté(?:e|s|es)? employé(?:e|s|es)? (?:à|de|du|au|en)|et parti pour aller (?:à|de|du|au|en)|pour aller (?:à|de|du|au|en)|et resté(?:e|s|es)? officier des troupes (?:à|de|du|au|en)|malade à la relache (?:à|de|du|au|en)|au comptoir (?:à|de|du|au|en)|et resté(?:e|s|es)? chez son oncle (?:à|de|du|au|en)|pour l\'hôpital (?:à|de|du|au|en)|par ordre du Commandant (?:à|de|du|au|en)|par ordre (?:à|de|du|au|en)|de gré à l\'hôpital (?:à|de|du|au|en)|et resté(?:e|es|s)? à l\'hôpital (?:à|de|du|au|en)|avec ses effets (?:à|de|du|au|en)|et resté(?:e|es|s)? à ses frais (?:à|de|du|au|en)|et resté(?:e|es|s)? malade à l\'hôpital (?:à|de|du|au|en)|et resté(?:e|es|s)? aumônier à l\'hôpital (?:à|de|du|au|en)|avec ses hardes, de gré à gré (?:à|de|du|au|en)|avec ses hardes (?:à|de|du|au|en)|ou resté(?:e|es|s)? (?:à|de|du|au|en)|et engagé(?:e|es|s)? dans la troupe (?:à|de|du|au|en)|dans la|et engagé(?:e|es|s)? dans le régiment (?:à|de|du|au|en)|et engagé(?:e|es|s)? dans|et engagé(?:e|es|s)? (?:à|de|du|au|en)|pour la traite (?:à|de|du|au|en)|et retenu (?:à|de|du|au|en)|pour maladie (?:à|de|du|au|en)|pour affaires de famille (?:à|de|du|au|en)|à malade à l\'hôpital (?:à|de|du|au|en)|et revenu sur le|malade dans le bateau du pilote (?:à|de|du|au|en)|malade dans le|et incorporé(?:e|es|s)? à la garnison (?:à|de|du|au|en)|et incorporé(?:e|es|s)? dans la garnison (?:à|de|du|au|en)|pour le service (?:à|de|du|au|en)|malade et resté(?:e|es|s)? (?:à|de|du|au|en)|de bonne volonté (?:à|de|du|au|en)|et mis à bord (?:à|de|du|au|en)|malade à la relâche (?:à|de|du|au|en)|à la relâche (?:à|de|du|au|en)|sans ordre (?:à|de|du|au|en)|et resté(?:e|es|s)? comme matelot (?:à|de|du|au|en)|malade à l\'hôpital du roi (?:à|de|du|au|en)|à l\'hôpital du roi (?:à|de|du|au|en)|au bas (?:à|de|du|au|en)|et (?:congédié(?:e|es|s)?|soldé(?:e|es|s)?) (?:à|de|du|au|en)|et remis sur le fort (?:à|de|du|au|en)|et remis (?:à|de|du|au|en)|pour se rendre (?:à|de|du|au|en)|blessé(?:e|s|es)? malade (?:à|de|du|au|en)|blessé(?:e|s|es)? (?:à|de|du|au|en)|de gré pour le service (?:à|de|du|au|en)|de gré à gré (?:à|de|du|au|en)|puis passé(?:e|es|s)? malade (?:à|de|du|au|en)|et resté(?:e|es|s)? (?:à|de|du|au|en)|et passé(?:e|es|s)? sur (?:le|la)|et passé(?:e|es|s)? (?:à|de|du|au|en)|et entré(?:e|es|s)? aux forges (?:à|de|du|au|en)|et mis au (?:prison|cachot) (?:à|de|du|au|en)|de son gré (?:à|de|du|au|en)|malade d\'une descente à la relâche (?:à|de|du|au|en)|malade du scorbut (?:à|de|du|au|en)|et resté(?:e|es|s)? (?:à|de|du|au|en)|dans le|et renvoyé(?:e|es|s)? (?:à|de|du|au|en)|et entré(?:e|es|s)? à l\'hôpital (?:à|de|du|au|en)|et payé(?:e|es|s)? (?:à|de|du|au|en)|de gré (?:à|de|du|au|en)|et resté(?:e|es|s)? à terre|au service du roi (?:à|de|du|au|en) |à l\'hôpital (?:à|de|du|au|en) |en échange (?:à|de|du|au|en) |furtivement (?:à|de|du|au|en) |malade après être tombé du haut mal (?:à|de|du|au|en) |malade à l\'hôpital (?:à|de|du|au|en) |malade et mort(?:e|es|s)? à l\'hôpital (?:à|de|du|au|en) |malade à l\'hôpital |malade (?:à|de|du|au|en) |à malade (?:à|de|du|au|en) |à malade |à |au |aux |en appareillant |en |de |du |malade |pour )?([\w\s\'-îÎ]+?)(?=( ---|---|\s+(?:le|du)\s+\d{2}-\d{2}/\d{4}|\s+(?:le|du)\s+\d{2}/\d{2}/\d{4}|en \d{4}|\d{2}/\d{2}/\d{4}|(?:le|du)\s+\d{2}/\d{2}/\d{4}|le \d{2}|avec sa femme|et passé(?:e|es|s)?|et remplacé(?:e|es|s)?|lors de|de bonne volonté|sur le|en (?:janvier|février|mars|avril|mai|juin|juillet|août|septembre|octobre|novembre|décembre)|à la demande|au service|à l\'hôpital|et repassé(?:e|es|s)? à|pour|par ordre|et parti|à la relâche|de gré|malade|(?:le|du)\s+\?|\n|$|et n\'a pas|pour aller|au retour|et passé(?:e|es|s)? sur|en appareillant|et remis sur|avec|et rendu|sans ordre|le fleuve|et resté(?:e|es|s)?|comme tonnelier|avant|entre|pendant|en allant|blessé|dans les|de son|chez les|et commandé|à l\'arrivé|au soir|et engagé|solde|incommodé|revenu|chez|par échange|décédé|et remis|le nommé|après|solde|par décision|\s*passé(?:e|es|s)?(?:s|e|es)?|échappé(?:e|s|es)?| échappé(?:e|s|es)?|mort(?:e|es|s)?(?:e|s|es)?| mort(?:e|es|s)?(?:e|s|es)?|noyé(?:e|s|es)?| noyé(?:e|s|es)?))'
        },
        'déserté': {
            'date': r'\bdéserté(?:e|es|s)?(?: à [\w\s\'\-îÎ]+| sur le vaisseau de côte le [\w\s\'\-îÎ]+| en [\w\s\'\-îÎ]+| au départ à [\w\s\'\-îÎ]+| au [\w\s\'\-îÎ]+)? (?:le |du )?(\d{2}/\d{2}/\d{4})',
            'location': r'\bdéserté(?:e|es|s)? (?:à bord (?:à|de|du|au|en)|à la relâche (?:à|de|du|au|en)|à l\'hôpital à la relâche|en venant (?:à|de|du|au|en)|à l\'hôpital (?:à|de|du|au|en)|à l\'hôpital|en rade (?:à|de|du|au|en)|à|en|au départ (?:à|de|du|au|en)|au départ (?:à|de|du|au|en)|au) ([\w\s\'-îÎ]+?)(?: le|\n|,| ---|$| au départ|\d{2}/\d{2}/\d{4}| avec| dans| avant| la nuit| et passé| du Fortuné| du sénault| à la relâche| après| et revenu| déclaration| et voleur| étant| en \d{4}| en (?:janvier|février|mars|avril|mai|juin|juillet|août|septembre|octobre|novembre|décembre))'
        },
        'mort en mer': {
            'date': r'\b(?:mort(?:e|es|s)?|noyé(?:e|es|s)?) (?:en mer |noyé en [\w\s\'-îÎ]+ |du [\w\s\'-îÎ]+ en [\w\s\'-îÎ]+ |à l\'hôpital de [\w\s\'-îÎ]+ |à la ration |à [\w\s\'-îÎ]+ |tombé à la mer et mort noyé |dans le Gange |au [\w\s\'-îÎ]+ |en [\w\s\'-îÎ]+ )?(?:le|le )(\d{2}/\d{2}/\d{4})',
            'location': r'\b(?:mort(?:e|es|s)?|noyé(?:e|es|s)?) (?:en mer|à l\'hôpital (?:du|de)|à la ration|à|au|en) ([\w\s\'-îÎ]+?)(?= le \d{2}/\d{2}/\d{4}| le\d{2}/\d{2}/\d{4}| ---| \n)'
        },
#         'a fait la': {
#             'date': r'levé(?:e|es|s)? du (\d{2}/\d{2}/\d{4})',
#             'location': r'\b(?:a|à) fait la campagne de [\w\s\'-îÎ]+? à ([\w\s\'-îÎ]+?)(?=\s*(?:du|le) \d{2}/\d{2}/\d{4}|\s*---|,|en|\n|$)'
#         },
        'passé': {
            'date': r'\bpassé(?:e|es|s)? (?:sur la [\w\s\'-îÎ]+ en rade (?:à|de|du|au|en)|à|de|du|au|en) [\w\s\'-îÎ]+ le (\d{2}/\d{2}/\d{4})',
            'location': r'\bpassé(?:e|es|s)? (?:sur la [\w\s\'-îÎ]+ en rade (?:à|de|du|au|en)|à|de|du|au|en) ([\w\s\'-îÎ]+)(?=\s+(?:le\s+\d{2}/\d{2}/\d{4}|,| ---|\n|$))'
        },
        'resté': {
            'date': r'\bresté(?:e|es|s)?(?:\s+(?:à la partance (?:à|de|du|au|en)|en fonction \n resté(?:es|e|s)? malade à l\'hôpital (?:à|de|du|au|en)|malade à l\'hôpital général (?:à|de|du|au|en)|à bord en sa qualité après le|à la relâche (?:à|de|du|au|en)|malade l\'hôpital (?:à|de|du|au|en)|à l\'hôpital du roi (?:à|de|du|au|en)|malade au départ à l\'hôpital (?:à|de|du|au|en)|au départ du vaiseau (?:à|de|du|au|en)|malade à l\'hôpital de la marine (?:à|de|du|au|en)|à l\'hôpital de la marine (?:à|de|du|au|en)|à terre en remplacement (?:à|de|du|au|en)|malade de folie (?:à|de|du|au|en)|malade à l\'hôpital du roi (?:à|de)|à terre à la relâche à|en remplacement à|à terre à l\'armement au|malade d\'un navire qui a relâché à|malade à l\'hôpital général (?:de|du) roi à|à terre sur la patache du|à bord jusqu\'à la prise à|malade au port du sud-est à|malade du scorbut à|à terre à l\'hôpital (?:de|du|à)|malade à terre à|malade à la relâche à|à terre malade à l\'hôpital (?:de|du)|malade à terre à|à terre malade en|à terre (?:à|du|en|au|de)|à terre|malade à l\'hôpital (?:de|du|à|au)|à l\'hôpital (?:à|du)|à l\'hôpital de|malade au départ de|à terre au départ de|à terre malade au départ de|à terre malade à|à|en|au départ de|malade à|malade au|malade)?(?: [\w\s\'-îÎ]+?)?)? (?:le|du)\s*(\d{2}/\d{2}/\d{4})(?:\b.*?| rejoint|\n|,| ---|$)',
            'location': r'\bresté(?:e|es|s)? (?:à la partance (?:à|de|du|au|en)|en fonction \n resté(?:es|e|s)? malade à l\'hôpital (?:à|de|du|au|en)|malade à l\'hôpital général (?:à|de|du|au|en)|à bord en sa qualité après le|à la relâche (?:à|de|du|au|en)|malade l\'hôpital (?:à|de|du|au|en)|à l\'hôpital du roi (?:à|de|du|au|en)|malade au départ à l\'hôpital (?:à|de|du|au|en)|au départ du vaiseau (?:à|de|du|au|en)|malade à l\'hôpital de la marine (?:à|de|du|au|en)|à l\'hôpital de la marine (?:à|de|du|au|en)|à terre en remplacement (?:à|de|du|au|en)|malade de folie (?:à|de|du|au|en)|malade à l\'hôpital du roi (?:à|de|du|au|en)|à terre à la relâche (?:à|de|du|au|en)|en remplacement (?:à|de|du|au|en)|à terre à l\'armement (?:à|de|du|au|en)|malade d\'un navire qui a relâché (?:à|de|du|au|en)|malade à l\'hôpital général (?:de|du) roi (?:à|de|du|au|en)|à terre sur la patache (?:à|de|du|au|en)|à bord jusqu\'à la prise (?:à|de|du|au|en)|malade au port du sud-est (?:à|de|du|au|en)|malade du scorbut (?:à|de|du|au|en)|à terre à l\'hôpital (?:à|de|du|au|en)|malade à terre (?:à|de|du|au|en)|malade à la relâche (?:à|de|du|au|en)|à terre malade à l\'hôpital (?:à|de|du|au|en)|à terre malade (?:à|de|du|au|en)|à terre (?:à|de|du|au|en)|à terre|malade à l\'hôpital (?:à|de|du|au|en)|malade à l\'hôpital|à l\'hôpital (?:à|de|du|au|en)|malade au départ (?:à|de|du|au|en)|à terre au départ (?:à|de|du|au|en)|à terre malade au départ (?:à|de|du|au|en)|à terre malade (?:à|de|du|au|en)|à|en|au départ (?:à|de|du|au|en)|malade (?:à|de|du|au|en)|malade) ([\w\s\'-îÎ]+?)(?: en supplément à| (?:du|le)\s*\d{2}/\d{2}/\d{4}|\d{2}/\d{2}/\d{4}|le \?| rejoint|\n|,| ---|---|$| au départ| tombé| blessé| du consentem| le navire| ne s\'est| de bonne volonté| vient| congédié| et congédié| compagnie| comme aumônier| domestique| fils| engagé| malade et remplacé| et remplacé| remplacé| et décédé| pour| à l\'office| n\'ayant pas| par ordre| après| à l\'hôpital| hospitalisé| au service du| malade le| et passé sur| de gré| au service| le dit| comme| malade à la relâche le| ayant| à la relâche| sans décompte| sous les| et a remboursé| avec| jusqu\'au| dans| nouvelle| malade| et revenu| sans permission| à compter| doit| où il| en (?:janvier|février|mars|avril|mai|juin|juillet|août|septembre|octobre|novembre|décembre))'
        }
    }

    # Initialize results
    embark_location = pd.NA
    embark_date = pd.NA
    disembark_location = pd.NA
    disembark_date = pd.NA

    # Initialize match variables
    embark_date_match = None
    embark_location_match = None
    disembark_date_match = None
    disembark_location_match = None

    # Determine which embark patterns to use
    if 'embarqué en remplacement' in text.lower():
        embark_date_match = re.search(embark_patterns['embanremp']['date'], text, flags=re.IGNORECASE)
        embark_location_match = re.search(embark_patterns['embanremp']['location'], text, flags=re.IGNORECASE)
    elif 'embarqué' in text.lower() or 'rembarqué' in text.lower():
        embark_date_match = re.search(embark_patterns['embarqué']['date'], text, flags=re.IGNORECASE)
        embark_location_match = re.search(embark_patterns['embarqué']['location'], text, flags=re.IGNORECASE)
#     elif 'fait la campagne' in text.lower():
#         embark_date_match = re.search(embark_patterns['a fait la']['date'], text, flags=re.IGNORECASE)
#         embark_location_match = re.search(embark_patterns['a fait la']['location'], text, flags=re.IGNORECASE)
    elif 'supplément' in text.lower():
        embark_date_match = re.search(embark_patterns['supplément']['date'], text, flags=re.IGNORECASE)
        embark_location_match = re.search(embark_patterns['supplément']['location'], text, flags=re.IGNORECASE)
    elif 'remplacement' in text.lower() or 'remplaçement' in text.lower():
        embark_date_match = re.search(embark_patterns['remplacement']['date'], text, flags=re.IGNORECASE)
        embark_location_match = re.search(embark_patterns['remplacement']['location'], text, flags=re.IGNORECASE)
    elif 'complément' in text.lower():
        embark_date_match = re.search(embark_patterns['complément']['date'], text, flags=re.IGNORECASE)
        embark_location_match = re.search(embark_patterns['complément']['location'], text, flags=re.IGNORECASE)        
    elif 'trouvé' in text.lower():
        embark_date_match = re.search(embark_patterns['trouvé']['date'], text, flags=re.IGNORECASE)
        embark_location_match = re.search(embark_patterns['trouvé']['location'], text, flags=re.IGNORECASE)        
    elif 'soldat à bord le' in text.lower():
        embark_date_match = re.search(embark_patterns['soldat']['date'], text, flags=re.IGNORECASE)
    elif re.search(r'\bné(?:e|es|s)?\b', text.lower()):
        embark_date_match = re.search(embark_patterns['né']['date'], text, flags=re.IGNORECASE)

    # Extract embark information
    if embark_date_match:
        embark_date = embark_date_match.group(1)
    if embark_location_match:
        embark_location = str(embark_location_match.group(1)).strip()

    # Determine which disembark patterns to use
    if 'débarqué' in text.lower():
        disembark_date_match = re.search(disembark_patterns['débarqué']['date'], text, flags=re.IGNORECASE)
        disembark_location_match = re.search(disembark_patterns['débarqué']['location'], text, flags=re.IGNORECASE)
    elif 'déserté' in text.lower():
        disembark_date_match = re.search(disembark_patterns['déserté']['date'], text, flags=re.IGNORECASE)
        disembark_location_match = re.search(disembark_patterns['déserté']['location'], text, flags=re.IGNORECASE)
#     elif 'fait la campagne' in text.lower():
#         disembark_date_match = re.search(disembark_patterns['a fait la']['date'], text, flags=re.IGNORECASE)
#         disembark_location_match = re.search(disembark_patterns['a fait la']['location'], text, flags=re.IGNORECASE)
    elif 'resté' in text.lower():
        disembark_date_match = re.search(disembark_patterns['resté']['date'], text, flags=re.IGNORECASE)
        disembark_location_match = re.search(disembark_patterns['resté']['location'], text, flags=re.IGNORECASE)
    elif 'passé' in text.lower():
        disembark_date_match = re.search(disembark_patterns['passé']['date'], text, flags=re.IGNORECASE)
        disembark_location_match = re.search(disembark_patterns['passé']['location'], text, flags=re.IGNORECASE)
    elif 'mort' in text.lower() or 'noyé' in text.lower():
        disembark_date_match = re.search(disembark_patterns['mort en mer']['date'], text, flags=re.IGNORECASE)
        disembark_location_match = re.search(disembark_patterns['mort en mer']['location'], text, flags=re.IGNORECASE)

    # Extract disembark information
    if disembark_date_match:
        disembark_date = disembark_date_match.group(1)
    if disembark_location_match:
        disembark_location = str(disembark_location_match.group(1)).strip()
#         if len(disembark_location_match.groups()) > 1 and disembark_location_match.group(2):
#             disembark_location += f", {disembark_location_match.group(2)}"
    if 'mort à bord' in text.lower() or 'morte à bord' in text.lower() or 'morts à bord' in text.lower() or 'mortes à bord' in text.lower():
        disembark_location = 'on board'
    elif 'mort en mer' in text.lower() or 'morte en mer' in text.lower() or 'morts en mer' in text.lower() or 'mortes en mer' in text.lower():
        disembark_location = 'on board'
    if 'née en mer' in text.lower() or 'né en mer' in text.lower() or 'nées en mer' in text.lower() or 'nés en mer' in text.lower():
        embark_location = 'on board'
    elif 'née à bord' in text.lower() or 'né à bord' in text.lower() or 'nées à bord' in text.lower() or 'nés à bord' in text.lower():
        embark_location = 'on board'
    
            

    return embark_location, embark_date, disembark_location, disembark_date

def fill_emb_loc_for_rembarque(df):
    # Sort the dataframe by Last Name, Emb_date, and Disemb_date
    df_sorted = df.sort_values(by=['Last Name', 'Emb_date', 'Disemb_date']).reset_index(drop=True)
    
    # Iterate through the sorted dataframe
    for i in range(1, len(df_sorted)):
        current_row = df_sorted.iloc[i]
        previous_row = df_sorted.iloc[i - 1]
        
        # Check if Remarks is a string and contains the keyword "rembarqué"
        if pd.isna(current_row['Emb_loc']) and isinstance(current_row['Remarks'], str) and 'rembarqué' in current_row['Remarks'].lower():
            # Set the current row's Emb_loc to the previous row's Disemb_loc
            df_sorted.at[i, 'Emb_loc'] = previous_row['Disemb_loc']
    
    return df_sorted

In [2]:
df = pd.read_csv('MOTHER_TABLE.csv').drop('Unnamed: 0', axis=1).dropna(how='all', axis=0).dropna(how='all', axis=1)
vcs = list(set(df['voyage_code']))
combined = []
errors = []
for code in tqdm(vcs):
    try:
        if isinstance(code, str) and code != 'nan':
            df = pd.read_csv('MOTHER_TABLE.csv').drop('Unnamed: 0', axis=1).dropna(how='all', axis=0).dropna(how='all', axis=1)
            voyage_code = code
            df = df[df['voyage_code']==voyage_code]
            # get the first location and last location of the whole voyage + dates
            steps_l = list(df.dropna(subset='steps').drop_duplicates(subset='steps')['steps'])
            if len(steps_l) > 0:
                steps = steps_l[0]
                stops = []
                if isinstance(steps, str):
                    clean = steps.replace(",", "-").replace("→", "-")
                    temp = re.split(r'(?<=\w)-(?=\s)|(?<=\s)-(?=\w)|(?<=\s)-(?=\s)', clean)
                    for j in temp:
                        stops.append(j.replace("?", "").strip())
                sl = stops[0]
                ll = stops[-1]
            else:
                sl_l = list(df.dropna(subset='start_loc').drop_duplicates(subset='start_loc')['start_loc'])
                ll_l = list(df.dropna(subset='last_loc').drop_duplicates(subset='last_loc')['last_loc'])
                if len(sl_l) > 0:
                    sl = sl_l[0]
                else:
                    sl = '?'
                if len(ll_l) > 0:
                    ll = ll_l[0]
                else:
                    ll = '?'
            dvb_l = list(df.dropna(subset='date_voy_begin').drop_duplicates(subset='date_voy_begin')['date_voy_begin'])
            dve_l = list(df.dropna(subset='date_voy_end').drop_duplicates(subset='date_voy_end')['date_voy_end'])
            if len(dvb_l) > 0:
                dvb = dvb_l[0]
            else:
                dvb = '11/11/1111'
            if len(dve_l) > 0:
                dve = dve_l[0]
            else:
                dve = '11/11/1111'
            df = df[['n', 'voyage_code', 'last_name', 'first_name', 'f_n', 'remarks', 'age', 'wage',
                   'emb_date', 'disemb_date', 'disemb_loc']].drop_duplicates()
            df.rename(columns={'last_name': 'Last Name', 'first_name': 'First Name', 'f_n': 'Function', 'remarks': 'Remarks', 'age': 'Age', 'wage': 'Wage',
                   'emb_date': 'Emb_date', 'disemb_date': 'Disemb_date', 'disemb_loc': 'Disemb_loc', 'voyage_code': 'Voyage Code'}, inplace=True)
            for col in df.select_dtypes(include=['object']):
                df[col] = df[col].str.strip()
            df.replace('[nan]', np.nan, inplace=True)
            df.replace('<Disemb_Loc>', np.nan, inplace=True)

            # split by re-embark and Extracting info from Remarks + re-embark
            df = process_rembarque(df)

            df['Emb_loc'] = None
            for index, row in df.iterrows():
                remark = re.sub(r'\s+', ' ', str(row['Remarks']).replace('\n', ' '))

                emb_date = row['Emb_date']
                disemb_date = row['Disemb_date']
                disemb_loc = row['Disemb_loc']
                embark_location, embark_date, disembark_location, disembark_date = extract_details(remark)

                df.at[index, 'Emb_loc'] = embark_location
                if pd.notna(disembark_date):
                    df.at[index, 'Disemb_date'] = disembark_date
                if pd.notna(embark_date):
                    df.at[index, 'Emb_date'] = embark_date
                if pd.notna(disembark_location):
                    df.at[index, 'Disemb_loc'] = disembark_location

            df = fill_emb_loc_for_rembarque(df)

            des = ll.strip()
            arm = sl.strip()
            # replacing "armement" and "désarmement"
            for index, row in df.iterrows():
                if pd.notna(row['Remarks']):
                    if isinstance(row['Disemb_loc'], str):
                        if 'désarmement' in row['Disemb_loc'].lower() or 'débarquement' in row['Disemb_loc'].lower() or row['Disemb_loc'].lower()=='désarmement':
                            df.at[index, 'Disemb_loc'] = des

                            if pd.isna(row['Disemb_date']):
                                df.at[index, 'Disemb_date'] = dve
                        elif 'armement' in row['Disemb_loc'].lower() or 'embarquement' in row['Disemb_loc'].lower():
                            df.at[index, 'Disemb_loc'] = arm

                    if isinstance(row['Emb_loc'], str):
                        if 'armement' in row['Emb_loc'].lower() or 'embarquement' in row['Emb_loc'].lower():
                            df.at[index, 'Emb_loc'] = arm

                            if pd.isna(row['Emb_date']):
                                df.at[index, 'Emb_date'] = dvb
                        elif 'désarmement' in row['Emb_loc'].lower() or 'débarquement' in row['Emb_loc'].lower() or row['Emb_loc'].lower()=='désarmement':
                            df.at[index, 'Emb_loc'] = des

                    if 'a fait la campagne' in row['Remarks'].lower():
                        if pd.isna(row['Emb_date']):
                            df.at[index, 'Emb_date'] = dvb
                        if pd.isna(row['Disemb_date']):
                            df.at[index, 'Disemb_date'] = dve
                        if pd.isna(row['Emb_loc']):
                            df.at[index, 'Emb_loc'] = arm
                        if pd.isna(row['Disemb_loc']):
                            df.at[index, 'Disemb_loc'] = des

            # datetype handling
            df['Emb_date'] = df['Emb_date'].apply(convert_to_ddmmyyyy)
            df['Disemb_date'] = df['Disemb_date'].apply(convert_to_ddmmyyyy)
            df['Emb_date'] = df['Emb_date'].apply(lambda x: x if isinstance(x, str) else np.nan)
            df['Disemb_date'] = df['Disemb_date'].apply(lambda x: x if isinstance(x, str) else np.nan)

            # Add emb_class and disemb_class based on keywords provided by Matt
            df['emb_class'], df['disemb_class'] = None, None
            for index, row in df.iterrows():
                if not isinstance(row['Remarks'], str):
                    continue
                if 'clandestin' in row['Remarks'] or 'caché' in row['Remarks'] or 'furtivement' in row['Remarks']:
                    df.at[index, 'emb_class'] = 304
                elif 'rembarqué' in row['Remarks']:
                    df.at[index, 'emb_class'] = 306
                elif 'fait la campagne' in row['Remarks'] or 'armement' in row['Remarks'] or 'embarqué' in row['Remarks']:
                    if pd.notna(row['Emb_loc']):
                        if row['Emb_loc'] == arm:
                            df.at[index, 'emb_class'] = 301
                        else:
                            df.at[index, 'emb_class'] = 302
                elif 'remplacement' in row['Remarks'] or 'supplément' in row['Remarks']:
                    df.at[index, 'emb_class'] = 302
                elif 'resté' in row['Remarks']:
                    df.at[index, 'emb_class'] = 308
                elif 'renversement' in row['Remarks'] or 'vient' in row['Remarks']:
                    df.at[index, 'emb_class'] = 303
                elif 'né' in row['Remarks']:
                    df.at[index, 'emb_class'] = 305
                else:
                    df.at[index, 'emb_class'] = 309

                if 'mort' in row['Remarks'] or 'noyé' in row['Remarks']:
                    df.at[index, 'disemb_class'] = 305
                elif 'fait la campagne' in row['Remarks']:
                    df.at[index, 'disemb_class'] = 301
                elif 'resté malade' in row['Remarks'] or 'resté à l\'hôpital' in row['Remarks'] or 'restée malade' in row['Remarks'] or 'restée à l\'hôpital' in row['Remarks']:
                    df.at[index, 'disemb_class'] = 306
                elif 'débarqué' in row['Remarks']:
                    if pd.notna(row['Disemb_loc']):
                        if row['Disemb_loc'] == des:
                            df.at[index, 'disemb_class'] = 301
                        else:
                            df.at[index, 'disemb_class'] = 302
                elif 'passé' in row['Remarks']:
                    df.at[index, 'disemb_class'] = 303
                elif 'échappé' in row['Remarks'] or 'fugitif' in row['Remarks'] or 'déserté' in row['Remarks']:
                    df.at[index, 'disemb_class'] = 304
                elif 'prise par le' in row['Remarks']:
                    df.at[index, 'disemb_class'] = 307
                elif 'resté' in row['Remarks']:
                    df.at[index, 'disemb_class'] = 308
                else:
                    df.at[index, 'disemb_class'] = 309

            reembark_rows = df[(df['Remarks'].str.contains('rembarqué', na=False, case=False)) & (df['Emb_loc'].isna())]

            # Filling remaining re-embarkation based on previous locations and dates
            for index, row in reembark_rows.iterrows():

                matching_rows = df[
                    match_or_nan(df['Last Name'], row['Last Name']) &
                    match_or_nan(df['First Name'], row['First Name']) &
                    match_or_nan(df['Function'], row['Function']) &
                    (df['Last Name']!='Anonymous')
                ]

                sorted_group = matching_rows.sort_values(by=['Emb_date', 'Disemb_date'])

                previous_disemb_loc = None
                for i, (sorted_index, sorted_row) in enumerate(sorted_group.iterrows()):
                    if sorted_index == index:
                        if previous_disemb_loc:
                            df.at[sorted_index, 'Emb_loc'] = previous_disemb_loc

                    previous_disemb_loc = sorted_row['Disemb_loc']

            for _, group in df.groupby(['Last Name', 'First Name', 'Function'], dropna=False):
                sorted_group = group.sort_values(by=['Emb_date', 'Disemb_date'])
                for i, (index, row) in enumerate(sorted_group.iterrows()):
                    if pd.isna(row['Disemb_loc']) and i < len(sorted_group) - 1:
                        if row['Last Name'] != 'Anonymous':
                            next_row = sorted_group.iloc[i + 1]
                            df.at[index, 'Disemb_loc'] = next_row['Emb_loc']

            df['Emb_date'] = df['Emb_date'].apply(convert_to_ddmmyyyy)
            df['Disemb_date'] = df['Disemb_date'].apply(convert_to_ddmmyyyy)
            df['Emb_date'] = df['Emb_date'].apply(lambda x: x if isinstance(x, str) else np.nan)
            df['Disemb_date'] = df['Disemb_date'].apply(lambda x: x if isinstance(x, str) else np.nan)
            df['Emb_loc'] = df['Emb_loc'].apply(lambda x: x if isinstance(x, str) else np.nan)
            df['Disemb_loc'] = df['Disemb_loc'].apply(lambda x: x if isinstance(x, str) else np.nan)
            df.replace('[nan]', np.nan, inplace=True)

            # handling cases where the subject is born or died during the voyage and the place and dates are not mentioned
            for index, row in df.iterrows():
                if isinstance(row['Remarks'], str) and ('mort' in row['Remarks'] or 'noyé' in row['Remarks']):
                    if pd.notna(row['Disemb_date']):
                        df.at[index, 'Disemb_loc'] = 'mort le '+row['Disemb_date']
                        df.at[index, 'Disemb_date'] = np.nan
                    else:
                        df.at[index, 'Disemb_loc'] = 'mort en mer'
                if isinstance(row['Remarks'], str) and re.search(r'\bné(?:e|s|es)?\b', row['Remarks']):
                    if pd.notna(row['Emb_date']):
                        df.at[index, 'Emb_loc'] = 'né le '+row['Emb_date']
                        df.at[index, 'Emb_date'] = np.nan
                    else:
                        df.at[index, 'Emb_loc'] = 'né en mer'
                # manually handling cases where the structure of the sentences were not proper (i.e. missing location but the sentence is structured in a way that there should be a location)
                exact = ['la 2e sortie', 'par accident', 'audit lieu', 'Lorient et trouvé caché à bord', 'navire', 'Mr Herman', '01/07/1774', 'sans décompte', 'un lieu inconnu', 'pour être patron de chaloupe', 'et remplacé par Jean Pierre ADAM de Lamballe,', 'ou débarqué ou', 'ou', 'et destitué', 'soldé au', 'et soldé', 'bord comme volontaire', 'l\'hôpital de la', 'roi', 'n\'a point paru', 'la fièvre tierce', 'rivière', 'la', 'passé', 'l\'hôpital revenu à bord', 'bord jusqu\'à l\'échouage', 'sous voiles en dehors du port', 'avec tout le détachement', 'fonction', 'suite', 'par congé', 'sans précisions', 'gré', 'nouveau', 'la relâche', 'chez lui malade', '30/03/1758', 'avec les avances', 'blessé', ' gré', '? ', 'le ?', '?', 'congédié', 'l\'hôpital', 'bord', 'le', 'terre', 'terrre', 'et', 'malade', 'par ordre']
                approx = ['pour l\'escadre de', 'janvier', 'allant', 'naviguer', 'avoir', 'caffres', 'désarmé', 'congédié', 'même jour', 'naufrage', 'rendu ses', 'terre', 'petite vérole', 'complément', 'remplacement', 'pour être employé', 'port le même', 'embarqué', 'pour cause', 'escale', 'départ', 'appareillant', 'puis', 'pris', 'prise', 'la prise par', 'sur', 'tombé du mal', '---', 'a servi', 'le lendemain', 'vaisseau']
                if isinstance(row['Emb_loc'], str):
                    for ins in approx:
                        if ins in row['Emb_loc']:
                            df.at[index, 'Emb_loc'] = np.nan
                    for ins in exact:
                        if str(row['Emb_loc']).strip()==ins:
                            df.at[index, 'Emb_loc'] = np.nan
                if isinstance(row['Disemb_loc'], str):
                    for ins in approx:
                        if ins in row['Disemb_loc']:
                            df.at[index, 'Disemb_loc'] = np.nan
                    for ins in exact:
                        if str(row['Disemb_loc']).strip()==ins:
                            df.at[index, 'Disemb_loc'] = np.nan
            df['Voyage Code'] = str(code)
            combined.append(df)
    except:
        print('************************************************', code)
        errors.append(code)

<ipython-input-2-d4b051d8eaca>:1: DtypeWarning: Columns (30,43,44,49) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('MOTHER_TABLE.csv').drop('Unnamed: 0', axis=1).dropna(how='all', axis=0).dropna(how='all', axis=1)
  0%|          | 0/1284 [00:00<?, ?it/s]<ipython-input-2-d4b051d8eaca>:8: DtypeWarning: Columns (30,43,44,49) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('MOTHER_TABLE.csv').drop('Unnamed: 0', axis=1).dropna(how='all', axis=0).dropna(how='all', axis=1)
<ipython-input-1-7bbb92484328>:126: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  return pd.to_datetime(date_value).strftime('%d/%m/%Y')
<ipython-input-1-7bbb92484328>:126: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inc

  1%|          | 10/1284 [00:55<2:14:07,  6.32s/it]<ipython-input-2-d4b051d8eaca>:8: DtypeWarning: Columns (30,43,44,49) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('MOTHER_TABLE.csv').drop('Unnamed: 0', axis=1).dropna(how='all', axis=0).dropna(how='all', axis=1)
<ipython-input-1-7bbb92484328>:126: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  return pd.to_datetime(date_value).strftime('%d/%m/%Y')
<ipython-input-1-7bbb92484328>:126: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  return pd.to_datetime(date_value).strftime('%d/%m/%Y')
  1%|          | 11/1284 [01:02<2:20:49,  6.64s/it]<ipython-input-2-d4b051d8eaca>:8: DtypeWarning: Columns (30,43,44,49) 

<ipython-input-1-7bbb92484328>:126: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  return pd.to_datetime(date_value).strftime('%d/%m/%Y')
  2%|▏         | 21/1284 [02:13<2:42:55,  7.74s/it]<ipython-input-2-d4b051d8eaca>:8: DtypeWarning: Columns (30,43,44,49) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('MOTHER_TABLE.csv').drop('Unnamed: 0', axis=1).dropna(how='all', axis=0).dropna(how='all', axis=1)
<ipython-input-1-7bbb92484328>:126: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  return pd.to_datetime(date_value).strftime('%d/%m/%Y')
<ipython-input-1-7bbb92484328>:126: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was

  2%|▏         | 31/1284 [03:19<2:33:07,  7.33s/it]<ipython-input-2-d4b051d8eaca>:8: DtypeWarning: Columns (30,43,44,49) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('MOTHER_TABLE.csv').drop('Unnamed: 0', axis=1).dropna(how='all', axis=0).dropna(how='all', axis=1)
<ipython-input-1-7bbb92484328>:126: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  return pd.to_datetime(date_value).strftime('%d/%m/%Y')
<ipython-input-1-7bbb92484328>:126: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  return pd.to_datetime(date_value).strftime('%d/%m/%Y')
  2%|▏         | 32/1284 [03:26<2:27:15,  7.06s/it]<ipython-input-2-d4b051d8eaca>:8: DtypeWarning: Columns (30,43,44,49) 

  3%|▎         | 41/1284 [09:27<17:41:28, 51.24s/it]<ipython-input-2-d4b051d8eaca>:8: DtypeWarning: Columns (30,43,44,49) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('MOTHER_TABLE.csv').drop('Unnamed: 0', axis=1).dropna(how='all', axis=0).dropna(how='all', axis=1)
<ipython-input-1-7bbb92484328>:126: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  return pd.to_datetime(date_value).strftime('%d/%m/%Y')
<ipython-input-1-7bbb92484328>:126: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  return pd.to_datetime(date_value).strftime('%d/%m/%Y')
  3%|▎         | 42/1284 [09:34<13:08:21, 38.08s/it]<ipython-input-2-d4b051d8eaca>:8: DtypeWarning: Columns (30,43,44,49

...

 97%|█████████▋| 1242/1284 [3:39:07<04:07,  5.89s/it]<ipython-input-2-d4b051d8eaca>:8: DtypeWarning: Columns (30,43,44,49) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('MOTHER_TABLE.csv').drop('Unnamed: 0', axis=1).dropna(how='all', axis=0).dropna(how='all', axis=1)
<ipython-input-1-7bbb92484328>:126: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  return pd.to_datetime(date_value).strftime('%d/%m/%Y')
<ipython-input-1-7bbb92484328>:126: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  return pd.to_datetime(date_value).strftime('%d/%m/%Y')
 97%|█████████▋| 1243/1284 [3:39:13<04:04,  5.97s/it]<ipython-input-2-d4b051d8eaca>:8: DtypeWarning: Columns (30,43,44,

<ipython-input-1-7bbb92484328>:126: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  return pd.to_datetime(date_value).strftime('%d/%m/%Y')
<ipython-input-1-7bbb92484328>:126: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  return pd.to_datetime(date_value).strftime('%d/%m/%Y')
 98%|█████████▊| 1252/1284 [3:40:07<03:14,  6.09s/it]<ipython-input-2-d4b051d8eaca>:8: DtypeWarning: Columns (30,43,44,49) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('MOTHER_TABLE.csv').drop('Unnamed: 0', axis=1).dropna(how='all', axis=0).dropna(how='all', axis=1)
<ipython-input-1-7bbb92484328>:126: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) w

 98%|█████████▊| 1261/1284 [3:41:04<02:21,  6.16s/it]<ipython-input-2-d4b051d8eaca>:8: DtypeWarning: Columns (30,43,44,49) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('MOTHER_TABLE.csv').drop('Unnamed: 0', axis=1).dropna(how='all', axis=0).dropna(how='all', axis=1)
<ipython-input-1-7bbb92484328>:126: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  return pd.to_datetime(date_value).strftime('%d/%m/%Y')
<ipython-input-1-7bbb92484328>:126: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  return pd.to_datetime(date_value).strftime('%d/%m/%Y')
 98%|█████████▊| 1262/1284 [3:41:10<02:13,  6.09s/it]<ipython-input-2-d4b051d8eaca>:8: DtypeWarning: Columns (30,43,44,

 99%|█████████▉| 1271/1284 [3:42:04<01:16,  5.92s/it]<ipython-input-2-d4b051d8eaca>:8: DtypeWarning: Columns (30,43,44,49) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('MOTHER_TABLE.csv').drop('Unnamed: 0', axis=1).dropna(how='all', axis=0).dropna(how='all', axis=1)
<ipython-input-1-7bbb92484328>:126: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  return pd.to_datetime(date_value).strftime('%d/%m/%Y')
<ipython-input-1-7bbb92484328>:126: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  return pd.to_datetime(date_value).strftime('%d/%m/%Y')
 99%|█████████▉| 1272/1284 [3:42:10<01:10,  5.83s/it]<ipython-input-2-d4b051d8eaca>:8: DtypeWarning: Columns (30,43,44,

100%|█████████▉| 1281/1284 [3:43:06<00:18,  6.33s/it]<ipython-input-2-d4b051d8eaca>:8: DtypeWarning: Columns (30,43,44,49) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('MOTHER_TABLE.csv').drop('Unnamed: 0', axis=1).dropna(how='all', axis=0).dropna(how='all', axis=1)
<ipython-input-1-7bbb92484328>:126: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  return pd.to_datetime(date_value).strftime('%d/%m/%Y')
<ipython-input-1-7bbb92484328>:126: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  return pd.to_datetime(date_value).strftime('%d/%m/%Y')
100%|█████████▉| 1282/1284 [3:43:12<00:12,  6.25s/it]<ipython-input-2-d4b051d8eaca>:8: DtypeWarning: Columns (30,43,44,

In [3]:
errors

['SYD01', 'AIG01', 'DPR01', 'ACH05']

In [4]:
combined_filled = pd.concat(combined)
combined_filled

,n,Voyage Code,Last Name,First Name,Function,Remarks,Age,Wage,Emb_date,Disemb_date,Disemb_loc,Emb_loc,emb_class,disemb_class
0,14.0,AME02,AGNAN,Pierre Antoine,NaN,a fait la campagne --- n'est pas sur le rôle d...,31.0,30,12/07/1727,20/06/1728,Lorient,Lorient,301,301
1,61.0,AME02,AUBIN,Guillaume,129.0,embarqué à l'armement \n débarqué à ? --- pour...,21.0,12,12/07/1727,5,NaN,Lorient,301,302
2,24.0,AME02,BABOUILLAN,Jérôme,88.0,a fait la campagne,25.0,14,12/07/1727,20/06/1728,Lorient,Lorient,301,301
3,25.0,AME02,BAILLE,Jean,27.0,a fait la campagne,23.0,13,12/07/1727,20/06/1728,Lorient,Lorient,301,301
4,1.0,AME02,BAILLIF,Jean Baptiste,NaN,a fait la campagne,30.0,150,12/07/1727,20/06/1728,Lorient,Lorient,301,301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,13.0,AST08,TRESENEC,Joseph,NaN,embarqué à l'armement \n débarqué au Sénégal l...,27.0,33,11/01/1752,23/02/1752,Sénégal,Lorient,301,302
104,54.0,AST08,TURYA DIT BEAUPLESSIS,Étienne,NaN,embarqué à Lorient \n débarqué au Sénégal le 0...,NaN,12,NaN,09/02/1752,Sénégal,Lorient,301,302
105,47.0,AST08,VALLER,René,NaN,embarqué à Lorient \n débarqué au Sénégal le 0...,NaN,NaN,NaN,09/02/1752,Sénégal,Lorient,301,302
106,108.0,AST08,VERAUX DIT VIVANDIER,Bertrand,NaN,remplacement au Sénégal le 23/02/1752 \n débar...,NaN,7.10,23/02/1752,NaN,NaN,Sénégal,302,302


In [5]:
combined_filled.to_csv('combined_filled.csv')

In [6]:
import pickle

with open('errors.pickle', 'wb') as f:
    pickle.dump(errors, f)